<a href="https://colab.research.google.com/github/kartikrupal/deep_learning/blob/main/expfour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

In [2]:
def get_global_configuration():
    """Retrieve configuration of the training process."""
    num_layers_to_add = 10
    return num_layers_to_add


In [4]:
def get_model_configuration():
    """Retrieve configuration for the model."""
    img_width, img_height = 32, 32
    no_classes = 10
    batch_size = 256
    #2^8
    no_epochs = 20
    validation_split = 0.2
    verbosity = 0
    return img_width, img_height, no_classes, batch_size, no_epochs, validation_split, verbosity


In [5]:
def get_dataset():
    """Load and preprocess dataset."""
    # Load relevant model configuration
    img_width, img_height, no_classes, _, _, _, _ = get_model_configuration()

    # Load CIFAR-10 dataset
    (input_train, target_train), (input_test, target_test) = cifar10.load_data()

    # Reshape and normalize data
    input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 3).astype('float32') / 255
    input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 3).astype('float32') / 255

    # Convert target labels to one-hot encoding
    target_train = tensorflow.keras.utils.to_categorical(target_train, no_classes)
    target_test = tensorflow.keras.utils.to_categorical(target_test, no_classes)

    return input_train, input_test, target_train, target_test

In [6]:
def create_base_model():
    """Create the base model."""
    _, _, no_classes, _, _, _, _ = get_model_configuration()

    model = Sequential()
    model.add(Flatten(input_shape=(32, 32, 3)))  # Define input shape for the first layer
    model.add(Dense(256, activation='relu'))
    model.add(Dense(no_classes, activation='softmax'))

    return model


In [7]:
def add_extra_layer(model):
    """Freeze existing layers, add a new dense layer, and retain the output layer."""
    layer_to_add = Dense(256, activation='relu')

    # Make all existing layers non-trainable
    for layer in model.layers:
        layer.trainable = False

    # Remove output layer
    output_layer = model.layers.pop()

    # Add new layer and re-add the output layer
    model.add(layer_to_add)
    model.add(output_layer)

    return model


In [8]:
def train_model(model, data):
    """Compile and train the model."""
    _, _, _, batch_size, no_epochs, validation_split, verbosity = get_model_configuration()

    input_train, input_test, target_train, target_test = data

    model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
                  optimizer=tensorflow.keras.optimizers.Adam(),
                  metrics=['accuracy'])

    # Train the model
    model.fit(input_train, target_train,
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity,
              validation_split=validation_split)

    # Evaluate the model
    score = model.evaluate(input_test, target_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

    return model

In [9]:
def training_process():
    """Run the training process."""
    model = create_base_model()
    data = get_dataset()

    num_layers_to_add = get_global_configuration()

    for _ in range(num_layers_to_add):
        model = train_model(model, data)  # Train before adding a layer
        model = add_extra_layer(model)  # Add an extra layer after training

# Run the training process
training_process()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 49s 0us/step
Test loss: 1.5309735536575317 / Test accuracy: 0.46129998564720154
Test loss: 1.5351516008377075 / Test accuracy: 0.46650001406669617
Test loss: 1.5711601972579956 / Test accuracy: 0.462799996137619
Test loss: 1.5974723100662231 / Test accuracy: 0.4625999927520752
Test loss: 1.6138209104537964 / Test accuracy: 0.46239998936653137
Test loss: 1.62672758102417 / Test accuracy: 0.46059998869895935
Test loss: 1.634934902191162 / Test accuracy: 0.4593000113964081
Test loss: 1.6417392492294312 / Test accuracy: 0.4571000039577484
Test loss: 1.6476757526397705 / Test accuracy: 0.45559999346733093
Test loss: 1.650026798248291 / Test accuracy: 0.4553999900817871
